# Push AOS Closed Loop Results to Chronograf

This notebook shows how to push a sample of results of the AOS closed loop to the [Chronograf dashboards](https://chronograf-demo.lsst.codes/) provided by the Squash team.

### Run WFS Zernike Estimation

This piece of code takes the PostISR output and runs the pieces of the closed loop that estimate Zernike polynomials from the donuts.

In [1]:
import os
import argparse
import numpy as np

from lsst.ts.wep.WepController import WepController
from lsst.ts.wep.Utility import CamType, FilterType, getModulePath, mapFilterRefToG, DefocalType
from lsst.ts.wep.ctrlIntf.WEPCalculationFactory import WEPCalculationFactory

In [30]:
run_deblender = False
save_postage_stamps = False
# Medium galactic latitude simulation
data_dir = '/astro/users/brycek/epyc/users/brycek/Commissioning/aos/ts_phosim/notebooks/analysis_scripts/test_output/gaiaMedClosedLoop/noDeblending/refCat/090320/'
postage_img_dir = os.path.join(data_dir, 'postage')

In [31]:
detector_list = ['R:2,2 S:0,0', 'R:2,2 S:0,1', 'R:2,2 S:0,2',
                 'R:2,2 S:1,0', 'R:2,2 S:1,1', 'R:2,2 S:1,2',
                 'R:2,2 S:2,0', 'R:2,2 S:2,1', 'R:2,2 S:2,2']

In [35]:
wep_calc = WEPCalculationFactory.getCalculator(CamType.ComCam, os.path.join(data_dir, 'input'))
wep_calc.wepCntlr.setPostIsrCcdInputs(os.path.join(data_dir, 'input/rerun/run1'))
intraObsId = 9006042
extraObsId = 9006041
visitList = [intraObsId, extraObsId]

In [39]:
# Remove existing database that might have table already created
if os.path.exists('/astro/users/brycek/epyc/users/brycek/Commissioning/aos/ts_wep/tests/testData/bsc.db3'):
    os.remove('/astro/users/brycek/epyc/users/brycek/Commissioning/aos/ts_wep/tests/testData/bsc.db3')

In [40]:
neighborStarMap = wep_calc._getTargetStar(visitList, defocalState=DefocalType.Intra)

isrImgMap = wep_calc.wepCntlr.getPostIsrImgMapByPistonDefocal(detector_list, visitList)

{'visit': 9006042, 'filter': 'g', 'raftName': 'R22', 'detectorName': 'S02'}
{'visit': 9006042, 'filter': 'g', 'raftName': 'R22', 'detectorName': 'S12'}
{'visit': 9006042, 'filter': 'g', 'raftName': 'R22', 'detectorName': 'S22'}
{'visit': 9006042, 'filter': 'g', 'raftName': 'R22', 'detectorName': 'S01'}
{'visit': 9006042, 'filter': 'g', 'raftName': 'R22', 'detectorName': 'S11'}
{'visit': 9006042, 'filter': 'g', 'raftName': 'R22', 'detectorName': 'S21'}
{'visit': 9006042, 'filter': 'g', 'raftName': 'R22', 'detectorName': 'S00'}
{'visit': 9006042, 'filter': 'g', 'raftName': 'R22', 'detectorName': 'S10'}
{'visit': 9006042, 'filter': 'g', 'raftName': 'R22', 'detectorName': 'S20'}
{'visit': 9006042, 'filter': 'g', 'raftName': 'R22', 'detectorName': 'S02'}
FITS standard SkyWcs:
Sky Origin: (232.788916, -4.239221)
Pixel Origin: (1555.02, 2128.17)
Pixel Scale: 0.200106 arcsec/pixel
0     232.889451
1     232.662075
2     232.878467
3     232.708678
4     232.858448
         ...    
78    232.88

In [41]:
donut_map = wep_calc.wepCntlr.getDonutMap(neighborStarMap, isrImgMap, FilterType.REF,
                                          doDeblending=run_deblender, postageImg=save_postage_stamps,
                                          postageImgDir=postage_img_dir)

donutMap = wep_calc.wepCntlr.calcWfErr(donut_map, postage_img_dir)


 Calculating the donut map 

 Calculating the wavefront error based on the donut map

 sensorName  R:2,2 S:0,2  abbrevDetectorName  R22_S02  starId= 0  donut px pos =  (3734.531372218985, 2244.1997855118602)

 sensorName  R:2,2 S:0,2  abbrevDetectorName  R22_S02  starId= 1  donut px pos =  (688.4249149630073, 1399.1831329428426)

 sensorName  R:2,2 S:1,2  abbrevDetectorName  R22_S12  starId= 46  donut px pos =  (797.9831279679986, 2553.5739347161293)

 sensorName  R:2,2 S:1,2  abbrevDetectorName  R22_S12  starId= 47  donut px pos =  (808.7182250841281, 822.1978465188822)

 sensorName  R:2,2 S:1,2  abbrevDetectorName  R22_S12  starId= 48  donut px pos =  (2521.4142978362397, 662.2715685118778)

 sensorName  R:2,2 S:2,2  abbrevDetectorName  R22_S22  starId= 91  donut px pos =  (249.33671853221108, 696.8152524007735)

 sensorName  R:2,2 S:2,2  abbrevDetectorName  R22_S22  starId= 92  donut px pos =  (1455.8860224751575, 2524.839447623227)

 sensorName  R:2,2 S:0,1  abbrevDetectorName  R2

In [76]:
listOfWfErr = wep_calc._populateListOfSensorWavefrontData(donutMap)
wfErrSensorsList = [int(wfErr.sensorId) for wfErr in listOfWfErr]

### Compare to OPD results and save

Here we load the OPD results from the same iteration of the closed loop

In [88]:
opd_data = np.genfromtxt(os.path.join(data_dir, 'iter4/img/opd.zer.1'))


In [91]:
listOfWfErr[0].annularZernikePoly

array([ 0.00040171, -0.02735912, -0.003069  , -0.03984362, -0.00503942,
        0.05550312, -0.01108701,  0.02334876, -0.00233945,  0.00655026,
        0.04078935,  0.00769696, -0.01027119,  0.01062344, -0.00568123,
        0.02074562, -0.01172678, -0.00201364, -0.01618952])

And load the dictionary that provides the sensor ID for each comcam sensor so we can compare the correct OPD array to the correct WFS array.

In [92]:
import yaml

In [93]:
with open('/astro/users/brycek/epyc/users/brycek/Commissioning/aos/ts_wep/policy/sensorNameToId.yaml', 'r') as f:
    
    sensor_id_dict = yaml.load(f, Loader=yaml.SafeLoader)

For this demonstration we will just compare the results of a single chip: `R22_S00`.

In [94]:
wfListIdx = np.where(int(sensor_id_dict['R22_S00']) == np.array(wfErrSensorsList))[0][0]

In [95]:
opd_data[0]

array([ 4.63216120e-02,  4.46069469e-02, -1.90879839e-01,  1.61867688e-04,
        1.72126167e-03,  5.14967927e-02, -5.82565825e-03,  3.26776449e-02,
        5.74246523e-03,  2.31919908e-03,  2.79290522e-02,  9.74587974e-03,
       -5.75359046e-03,  3.73554839e-03,  6.13784212e-04, -1.65242072e-02,
        4.84057889e-03,  3.99768909e-02, -1.16724148e-02])

In [96]:
listOfWfErr[wfListIdx].annularZernikePoly

array([ 0.02238296,  0.03540709, -0.04409899, -0.01786798, -0.00701338,
        0.02267609,  0.01745512,  0.02573043,  0.00443647,  0.0088546 ,
       -0.00320278,  0.00750134, -0.00899935, -0.00774009, -0.00878884,
        0.01899573, -0.00252246,  0.00494826, -0.01500698])

In [106]:
zerDiffDict = {}

Finally let's calculate the RMS across all the Zernike polynomials as a test value for this demonstration.

In [158]:
zerDiff = np.sqrt(np.mean(np.power(opd_data[0] - listOfWfErr[wfListIdx].annularZernikePoly, 2)))

In [108]:
zerDiffDict['S00'] = zerDiff

### Implement within `lsst_verify`

In [122]:
import lsst.verify
import astropy.units as u

For this to work some behind the scenes work is necessary. Every metrics package used with `lsst.verify` needs metrics defined and specs defined to test against. These are `yaml` files that go into a folder with the metric package name. Here we call this package `verify_aos` and add the necessary files in a folder of the same name.

### Create required metrics and specs files

#### Metric file

Here are the contents of the metric file:

In [176]:
! more verify_aos/metrics/wfs_zernike_compare.yaml

R22_S00:
  unit: nm
  description:
    RMS between this chip's WFS estimated Zernikes and the OPD values.
  reference:
    url: https://arxiv.org/pdf/1506.04839.pdf
  tags:
    - aos


#### Specs file

Here are the contents of the specs file:

In [177]:
! more verify_aos/specs/wfs_zernike_compare/specs.yaml

name: "design"
metric: "R22_S00"
threshold:
  operator: "<="
  unit: "nm"
  value: 100
tags:
  - "design"


### Load metrics package and run

In [173]:
METRIC_PACKAGE = "verify_aos"
metrics = lsst.verify.MetricSet.load_metrics_package(METRIC_PACKAGE)
specs = lsst.verify.SpecificationSet.load_metrics_package(METRIC_PACKAGE)

Once we load the metrics and specs files we created we can see that all the information gets wrapped within the respective objects.

In [136]:
metrics

Name,Description,Units,Reference,Tags
str27,str66,str13,str36,str3
wfs_zernike_compare.R22_S00,RMS between this chip's WFS estimated Zernikes and the OPD values.,$\mathrm{nm}$,https://arxiv.org/pdf/1506.04839.pdf,aos


In [137]:
specs

Name,Test,Tags
str34,str26,str6
wfs_zernike_compare.R22_S00.design,$x$ <= 100.0 $\mathrm{nm}$,design


Finally assign our measurement to the defined measurement from the `metrics` file. 

Note that this measurement needs to include the proper units from astropy.

In [159]:
s00_meas = lsst.verify.Measurement('wfs_zernike_compare.R22_S00', 1000.*zerDiffDict['S00']*u.nm)

In [161]:
job = lsst.verify.Job(metrics=metrics, specs=specs)
job.measurements.insert(s00_meas)

We can now run the job and print out a report.

In [162]:
job.report().show()

Status,Specification,Measurement,Test,Metric Tags,Spec. Tags
✅,wfs_zernike_compare.R22_S00.design,38.3 $\mathrm{nm}$,$x$ <= 100.0 $\mathrm{nm}$,N/A,design


### Push to Chronograf

In [ ]:
squash_api_url = "https://squash-restful-api-sandbox.lsst.codes"

In [ ]:
import getpass
username = getpass.getuser()
password = getpass.getpass(prompt='Password for user `{}`: '.format(username))

In [ ]:
import requests
credentials = {'username': username, 'password': password}

If this is your first time you can register as a new user by uncommenting the lines below.

In [ ]:
# r = requests.post('{}/register'.format(squash_api_url), json=credentials)
# r.json()

In [ ]:
r = requests.post('{}/auth'.format(squash_api_url), json=credentials)
r.json()

In [ ]:
headers = {'Authorization': 'JWT {}'.format(r.json()['access_token'])}

The following two cells upload the metric definitions to Squash and are a one-time setup procedure.

In [169]:
r = requests.post('{}/metrics'.format(squash_api_url),
                json={'metrics': metrics.json},
                headers=headers)
r.json()

{'message': 'A metric with name `wfs_zernike_compare.R22_S00` already exist.'}

In [170]:
r = requests.post('{}/specs'.format(squash_api_url),
                json={'specs': specs.json},
                headers=headers)
r.json()

{'message': 'A specification with name `wfs_zernike_compare.R22_S00.design` already exist.'}

Here we add some more metadata that is required for Squash.

In [171]:
job.meta.update({'packages': {}})
job.meta.update({'env': {'env_name': 'jenkins'}})

Finally, we dispatch the results of the `Job` we ran to Squash and can view them on the Squash dashboards.

In [172]:
job.dispatch(api_user=username, api_password=password, api_url=squash_api_url)